# Gray Matter CoreTech Sample data

Instructions:
Steps to compute measure score:
1. Select a measure
2. Filter records with diagnosis_codes that corresponds to the selected measure
Refer to the Diagnosis Codes table for diagnosis_codes for a given measure
3. Calculate comorbidity value for each row depending on set of comorbidity columns for the selected measure
Comorbidity value for a row can be calculated as count of ‘yes’ in corresponding comorbidity column set.Refer to the Comorbidity Columns table to find comorbidity columns for a given measure
4. Calculate lace score for each row in the dataset as sum of points for each LACE variable
Refer to the Lace Index table to find lace point for a given LACE variable
5. Select count of records as denominator
6. Select count of records with lace score > 9 as numerator
7. Calculate score for selected measure as numerator/denominator


# Initial Data wrangling

import pandas, read in csv
print data columns

In [9]:
import pandas as pd

data = pd.read_csv("SampleData2016.csv")
data.head()

,encounter_id,patient_nbr,race,gender,age,LengthofStay,ED_visits,Inpatient_visits,diagnosis_code,HistoryofPTCA,...,Urinarytractinfection,Otherinjuries,Skeletaldeformities,Posttraumaticosteoarthritis,Morbidobesity,Hypertension,Majorsymptomsabnormalities,HistoryofMechanicalVentilation,SleepApnea,OtherandUnspecifiedHeartDisease
0,2278392.0,8222157.0,Caucasian,Female,67.0,1.0,0.0,0.0,410.11,No,...,No,No,No,No,No,No,Yes,Yes,No,No
1,149190.0,55629189.0,Caucasian,Female,78.0,3.0,0.0,0.0,404.91,No,...,No,No,No,No,No,No,No,No,No,No
2,64410.0,86047875.0,AfricanAmerican,Female,86.0,2.0,5.0,1.0,CCS,No,...,No,Yes,No,No,Yes,No,No,Yes,No,Yes
3,500364.0,82442376.0,Caucasian,Male,65.0,2.0,1.0,0.0,402.01,No,...,No,Yes,No,No,No,No,No,No,No,Yes
4,16680.0,42519267.0,Caucasian,Male,60.0,1.0,0.0,0.0,402.11,No,...,No,Yes,No,No,Yes,No,No,No,No,Yes


In [11]:
columnNameList = data.columns.values.tolist()
print("data type: {}".format(type(data)))
print("data columns:")
for columnName in columnNameList:
    print(columnName)

data type: <class 'pandas.core.frame.DataFrame'>
data columns:
encounter_id
patient_nbr
race
gender
age
LengthofStay
ED_visits
Inpatient_visits
diagnosis_code
HistoryofPTCA
HistoryofCABG
Congestiveheartfailure
Acutecoronarysyndrome
Anteriormyocardialinfarction
Otherlocationofmyocardialinfarction
Anginapectorisoldmyocardialinfarction
Coronaryatherosclerosis
Valvularorrheumaticheartdisease
Specifiedarrhythmias
Historyofinfection
Metastaticcanceroracuteleukemia
Cancer
Diabetesmellitus(DM)orDMcomplications
Protein-caloriemalnutrition
Disordersoffluidelectrolyteacid-base
Irondeficiencyorotheranemiasandblooddisease
Dementiaorotherspecifiedbraindisorders
Hemiplegiaparaplegiaparalysisfunctionaldisability
Stroke
Cerebrovasculardisease
Vascularorcirculatorydisease
Chronicobstructivepulmonarydisease
Asthma
Pneumonia
End-stagerenaldiseaseordialysis
Renalfailure
Otherurinarytractdisorders
Decubitusulcerorchronicskinulcer
LungUpperDigestiveTractandOtherSevereCancers
LymphaticHeadandNeckBrainandOther

# Game plan

approach 1:
diagnosis codes, comorbidity values are like tables in rdb, could query to get values we want, then run program with values from those queries
approach 2:
quick and dirty, just put diagnosis codes, comorbidity values in csv, read them, and use them to run program

In [45]:
#get first row, look at types and values
row0 = data.iloc[0]
print("row0 type: {}".format(type(row0)))
for idx, element in enumerate(row0):
    print("idx: {} \tval: {} \ttype: {} \tcol: {}".format(idx, element, type(element), columnNameList[idx]))


row0 type: <class 'pandas.core.series.Series'>
idx: 0 	val: 2278392.0 	type: <class 'numpy.float64'> 	col: encounter_id
idx: 1 	val: 8222157.0 	type: <class 'numpy.float64'> 	col: patient_nbr
idx: 2 	val: Caucasian 	type: <class 'str'> 	col: race
idx: 3 	val: Female 	type: <class 'str'> 	col: gender
idx: 4 	val: 67.0 	type: <class 'numpy.float64'> 	col: age
idx: 5 	val: 1.0 	type: <class 'numpy.float64'> 	col: LengthofStay
idx: 6 	val: 0.0 	type: <class 'numpy.float64'> 	col: ED_visits
idx: 7 	val: 0.0 	type: <class 'numpy.float64'> 	col: Inpatient_visits
idx: 8 	val: 410.11 	type: <class 'str'> 	col: diagnosis_code
idx: 9 	val: No 	type: <class 'str'> 	col: HistoryofPTCA
idx: 10 	val: No 	type: <class 'str'> 	col: HistoryofCABG
idx: 11 	val: No 	type: <class 'str'> 	col: Congestiveheartfailure
idx: 12 	val: No 	type: <class 'str'> 	col: Acutecoronarysyndrome
idx: 13 	val: No 	type: <class 'str'> 	col: Anteriormyocardialinfarction
idx: 14 	val: No 	type: <class 'str'> 	col: Otherlocati

what's up with column 39? LymphaticHeadandNeckBrainandOtherMajorCancers;BreastColorectalandotherCancersandTumors;OtherRespiratoryandHeartNeoplasms
why does it have semicolons in it? is it a catch all for 3 types or is this a typo?

In [28]:
#verify that column name list length and row 0 are exactly equal:
if(len(columnNameList)!= len(row0)):
    raise ValueError("number of column labels not equal to number data columns ")
else:
    print("number of data columns = {}".format(len(columnNameList)))

number of data columns = 82


In [47]:
#pandas import mismatched header/data column test
badData = pd.read_csv("bad.csv")
print(badData.head())
print("shape: {}".format(badData.shape))
print("type: {}".format(type(badData)))
for element in badData:
    print("{} \ttype:{}".format(element, type(element)))

bdrow0 = badData.iloc[0]
print(len(bdrow0))
for element in bdrow0:
    print("{} \ttype:{}".format(element, type(element)))

     first_name last_name;greeting
Hans      Weeks              hello
shape: (1, 2)
type: <class 'pandas.core.frame.DataFrame'>
first_name 	type:<class 'str'>
last_name;greeting 	type:<class 'str'>
2
Weeks 	type:<class 'str'>
hello 	type:<class 'str'>


badData.csv should have 3 columns but instead it has 2! pandas does not throw any warnings about this mismatch